In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from shutil import copyfile, move
from tqdm import tqdm
import h5py

In [3]:
# train data
train_df = pd.read_csv('../input/aerial-cactus-identification/train.csv')
train_df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [4]:

from zipfile import ZipFile

with ZipFile('../input/aerial-cactus-identification/train.zip', 'r') as zipObj:
    zipObj.extractall()


with ZipFile('../input/aerial-cactus-identification/test.zip', 'r') as zipObj:
    zipObj.extractall()

In [5]:
train_df = pd.read_csv('../input/aerial-cactus-identification/train.csv', dtype=str)
test_df = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv', dtype=str)

In [6]:
from glob import glob

train_dir = np.array(glob('../working/train/'))
test_dir = np.array(glob('../working/test/'))

In [7]:
id_value = train_df['id'].values
id_value_test = test_df['id'].values

In [8]:
train_path = train_dir + id_value
test_path = test_dir + id_value_test

In [9]:
class_list = ['0', '1']
classes = np.array(train_df['has_cactus']).reshape([len(train_df['has_cactus']),-1])
onehot_classes = np.array(np.array(classes) == class_list, dtype = np.uint8)
onehot_classes

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [14]:
def get_onehot(path):
    cls_name = np.unique(train_df['has_cactus'])
    return tf.cast(tf.constant(classes) == cls_name, dtype=tf.uint8)

def read_image(path):
    image_raw = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image_raw)
    return image

def get_data(path):
    image = read_image
    onehot = get_onehot(path)
    return image, onehot

def get_data_onehot(path, onehot):
    image = read_image(path)
    return image, onehot


# from_tensor_slices
dataset = tf.data.Dataset.from_tensor_slices((train_path, onehot_classes))
dataset = dataset.map(get_data_onehot)
dataset = dataset.batch(64)
dataset = dataset.shuffle(100)


In [15]:
from tensorflow.keras import layers
input_shape = (32, 32, 3)
inputs = layers.Input(input_shape)
#Feature Extraction
net = layers.Conv2D(32, (3, 3), padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D((2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(64, (3, 3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D((2, 2))(net)
net = layers.Dropout(0.25)(net)

#Fully connected layers
net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.25)(net)
net = layers.Dense(2)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net, name = 'Basic_CNN')

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [17]:
model.fit(dataset, epochs=5)

Epoch 1/5


NotFoundError:  ../working/train0004be2cfeaba1c0361d39e2b000257b.jpg; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_1577]

Function call stack:
train_function
